In [4]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import math

# Map and player variables
map_size = 10
initial_player_pos = [0, 2, -5]  # Initial position of the player (Z is higher)
player_pos = list(initial_player_pos)  # X, Y, Z position of the player
player_velocity = [0, 0, 0]  # Movement velocity including gravity
player_look_angle = [0, 0]  # Rotation in x and y angles for mouse movement

# Constants
gravity = 0.05
jump_strength = 0.2
speed = 0.1

# Vertices of the cube
vertices = [
    [1, 0, -1],
    [1, 2, -1],
    [-1, 2, -1],
    [-1, 0, -1],
    [1, 0, 1],
    [1, 2, 1],
    [-1, 2, 1],
    [-1, 0, 1]
]

# Edges of the cube (to connect the vertices)
edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),  # Back face edges
    (4, 5), (5, 6), (6, 7), (7, 4),  # Front face edges
    (0, 4), (1, 5), (2, 6), (3, 7)   # Connecting edges
]

def draw_cube():
    glBegin(GL_LINES)
    for edge in edges:
        for vertex in edge:
            glVertex3fv(vertices[vertex])
    glEnd()

def draw_plane():
    # Draw a 3D plane as the ground
    glBegin(GL_QUADS)
    glColor3f(0.3, 0.6, 0.3)  # Green color for ground
    for x in range(-map_size, map_size):
        for y in range(-map_size, map_size):
            glVertex3f(x, 0, y)  # Ground level Z = 0
            glVertex3f(x + 1, 0, y)
            glVertex3f(x + 1, 0, y + 1)
            glVertex3f(x, 0, y + 1)
    glEnd()

    # Draw grid lines on top of the plane
    glColor3f(0.7, 0.7, 0.7)  # Light grey color for the grid
    glBegin(GL_LINES)
    
    # Draw vertical lines (X-axis)
    for x in range(-map_size, map_size + 1):
        glVertex3f(x, 0, -map_size)
        glVertex3f(x, 0, map_size)
    
    # Draw horizontal lines (Y-axis)
    for y in range(-map_size, map_size + 1):
        glVertex3f(-map_size, 0, y)
        glVertex3f(map_size, 0, y)
    
    glEnd()

def init_gl():
    glClearColor(0, 0.5, 1, 0)  # Sky blue background
    glEnable(GL_DEPTH_TEST)     # Enable depth test for 3D
    glMatrixMode(GL_PROJECTION)
    gluPerspective(60, (800 / 600), 0.1, 100.0)  # Increased far plane distance

def handle_mouse_movement():
    # Get the relative mouse movement for view rotation
    mouse_movement = pygame.mouse.get_rel()
    player_look_angle[0] += mouse_movement[0] * 0.1  # Left/Right rotation
    player_look_angle[1] += mouse_movement[1] * 0.1  # Inverted Up/Down rotation
    
    # Limit the vertical look angle (y-axis) so the player can't flip over
    player_look_angle[1] = max(-90, min(90, player_look_angle[1]))

def update_player_movement():
    keys = pygame.key.get_pressed()
    
    # Apply gravity
    player_velocity[2] -= gravity

    # Movement: WASD for forward, backward, left, right
    if keys[pygame.K_w]:
        player_pos[0] += math.sin(math.radians(player_look_angle[0])) * speed
        player_pos[1] += math.cos(math.radians(player_look_angle[0])) * speed
    if keys[pygame.K_s]:
        player_pos[0] -= math.sin(math.radians(player_look_angle[0])) * speed
        player_pos[1] -= math.cos(math.radians(player_look_angle[0])) * speed
    if keys[pygame.K_a]:
        player_pos[0] -= math.cos(math.radians(player_look_angle[0])) * speed
        player_pos[1] += math.sin(math.radians(player_look_angle[0])) * speed
    if keys[pygame.K_d]:
        player_pos[0] += math.cos(math.radians(player_look_angle[0])) * speed
        player_pos[1] -= math.sin(math.radians(player_look_angle[0])) * speed

    # Jump (Space)
    if keys[pygame.K_SPACE] and player_pos[2] <= 1.1:  # Allow jump if on ground
        player_velocity[2] = jump_strength

    # Reset player position (R key)
    if keys[pygame.K_r]:
        reset_player_position()
    
    # Update player position by velocity
    player_pos[2] += player_velocity[2]

    # Stop the player from falling below the ground
    if player_pos[2] < 1:
        player_pos[2] = 1
        player_velocity[2] = 0

def reset_player_position():
    # Reset player's position and velocity
    global player_pos, player_velocity, player_look_angle
    player_pos = list(initial_player_pos)
    player_velocity = [0, 0, 0]
    player_look_angle = [0, 0]

def apply_camera():
    # Apply camera rotation and position based on player's look and movement
    glRotatef(player_look_angle[1], 1, 0, 0)  # Vertical rotation
    glRotatef(player_look_angle[0], 0, 0, 1)  # Horizontal rotation
    glTranslatef(-player_pos[0], -player_pos[1], -player_pos[2])  # Translate for player movement

def main():
    pygame.init()
    display = (800, 600)
    pygame.display.set_mode(display, DOUBLEBUF | OPENGL)
    pygame.mouse.set_visible(False)  # Hide the mouse for first-person control
    pygame.mouse.set_pos(display[0] // 2, display[1] // 2)  # Center mouse
    pygame.event.set_grab(True)  # Capture mouse input for smooth camera movement

    init_gl()

    # Main loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        handle_mouse_movement()
        update_player_movement()

        # Clear the screen and draw the scene
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glLoadIdentity()
        apply_camera()
        draw_plane()  # Draw the plane with grid lines

        pygame.display.flip()
        pygame.time.wait(10)

    pygame.quit()

if __name__ == "__main__":
    main()


In [1]:
!pip install PyOpenGL PyOpenGL_accelerate pygame

import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import math


# Constants
speed = 0.1  # Camera movement speed
sensitivity = 0.2  # Mouse sensitivity
jump_strength = 0.3  # Strength of the jump
gravity = 0.01  # Gravitational pull on the player
ground_level = 1.75  # Y position of the ground

# Vertices of the cube
vertices = [
    [1, 0, -1],
    [1, 2, -1],
    [-1, 2, -1],
    [-1, 0, -1],
    [1, 0, 1],
    [1, 2, 1],
    [-1, 2, 1],
    [-1, 0, 1]
]

# Edges of the cube (to connect the vertices)
edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),  # Back face edges
    (4, 5), (5, 6), (6, 7), (7, 4),  # Front face edges
    (0, 4), (1, 5), (2, 6), (3, 7)   # Connecting edges
]

def draw_cube():
    glBegin(GL_LINES)
    for edge in edges:
        for vertex in edge:
            glVertex3fv(vertices[vertex])
    glEnd()

map_size = 10

def draw_plane():
    # Draw a 3D plane as the ground
    glBegin(GL_QUADS)
    glColor3f(0.3, 0.6, 0.3)  # Green color for ground
    for x in range(-map_size, map_size):
        for y in range(-map_size, map_size):
            glVertex3f(x, 0, y)  # Ground level Z = 0
            glVertex3f(x + 1, 0, y)
            glVertex3f(x + 1, 0, y + 1)
            glVertex3f(x, 0, y + 1)
    glEnd()

    # Draw grid lines on top of the plane
    glColor3f(0.7, 0.7, 0.7)  # Light grey color for the grid
    glBegin(GL_LINES)
    
    # Draw vertical lines (X-axis)
    for x in range(-map_size, map_size + 1):
        glVertex3f(x, 0, -map_size)
        glVertex3f(x, 0, map_size)
    
    # Draw horizontal lines (Y-axis)
    for y in range(-map_size, map_size + 1):
        glVertex3f(-map_size, 0, y)
        glVertex3f(map_size, 0, y)
    
    glEnd()
    
def main():
    pygame.init()
    display = (800, 600)
    pygame.display.set_mode(display, DOUBLEBUF | OPENGL)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45, (display[0] / display[1]), 0.1, 100.0)  # Perspective projection

    # Switch to ModelView Matrix (for object transformations)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    # glTranslatef(0.0, -1.0, -5)  # Move back so we can see the cube

    camera = Camera()
    
    pygame.mouse.set_visible(False)  # Hide the mouse for first-person control
    pygame.mouse.set_pos(display[0] // 2, display[1] // 2)  # Center mouse
    pygame.event.set_grab(True)  # Capture mouse input for smooth camera movement

    # Main loop
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
        keys = pygame.key.get_pressed()

        keys = pygame.key.get_pressed()
        if keys[K_w]:
            camera.move('FORWARD')
        if keys[K_s]:
            camera.move('BACKWARD')
        if keys[K_a]:
            camera.move('LEFT')
        if keys[K_d]:
            camera.move('RIGHT')

        if keys[K_SPACE]:
            camera.jump()

        # Get mouse movement for camera rotation
        mouse_dx, mouse_dy = pygame.mouse.get_rel()
        camera.rotate(mouse_dx, mouse_dy)
        camera.update_jump()

        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

        # Apply the camera transformations (first-person view)
        glLoadIdentity()
        camera.apply()

        draw_plane()
        draw_cube()
        pygame.display.flip()
        pygame.time.wait(10)

if __name__ == "__main__":
    main()


  Using cached PyOpenGL-accelerate-3.1.7.tar.gz (562 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build PyOpenGL_accelerate
pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


  error: subprocess-exited-with-error
  
  × Building wheel for PyOpenGL_accelerate (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [180 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\OpenGL_accelerate
      copying OpenGL_accelerate\__init__.py -> build\lib.win-amd64-cpython-312\OpenGL_accelerate
      running build_ext
      Compiling src\wrapper.pyx because it changed.
      [1/1] Cythonizing src\wrapper.pyx
      C:\Users\KIST-Robotics AI\AppData\Local\Temp\pip-build-env-avfwbk_p\overlay\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: C:\Users\KIST-Robotics AI\AppData\Local\Temp\pip-install-q2_6u5h9\pyopengl-accelerate_c48516b449ec44dab49a2162edbed13c\OpenGL_accelerate\wrapper.pxd
        tree = Parsing.p_module(s, pxd, full_module_name)
      buildin

error: video system not initialized

: 